In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df = pd.read_csv("breast-cancer.csv")
print(df.head())

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  radius_worst  texture_worst  perimeter_worst  area_wor

In [3]:
print(f"The total number of Columns is {df.shape[1]} and the total number of Rows is {df.shape[0]}.")

The total number of Columns is 32 and the total number of Rows is 569.


In [4]:
df.drop(columns=['id'],inplace=True)

In [5]:
print(f"The total number of Columns is {df.shape[1]} and the total number of Rows is {df.shape[0]}.")

The total number of Columns is 31 and the total number of Rows is 569.


In [6]:
df['diagnosis'].unique()

array(['M', 'B'], dtype=object)

In [7]:
x = df.iloc[:,1:] #All Colum Excepted 'diagnosis'
y = df.iloc[:,0] # 'diagnosis' Column

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [10]:
#Conver the all numpy Arrays to the Tensors.
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [11]:
print(x_test_tensor.shape)

torch.Size([114, 30])


In [12]:
class MyfirstNN():
    def __init__(self,x):
        self.weights = torch.rand(x.shape[1],dtype=torch.float64,requires_grad=True)
        self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)   

    def forward(self,x):
        z = torch.matmul(x,self.weights) + self.bias
        y_predict = torch.sigmoid(z)
        return y_predict

    def loss_function(self,y_pred,y):
        #Clamp Predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

        #calculate loss
        loss = -(y_train_tensor*torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss

In [13]:
learning_rate = 0.1
epochs = 20

In [14]:
#Create Model
model = MyfirstNN(x_train_tensor)

#Define Loop
for i in range(epochs): 
    
    #Forward Pass 
    y_predict = model.forward(x_train_tensor)
    
    #Loss Calculation
    loss = model.loss_function(y_predict,y_train_tensor)
    
    #Backward Pass
    loss.backward()
    
    #Parameters Updates.
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    #zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    #Print Loss in each epoch
    print(f"Epoch: {i + 1}, Loss : {loss.item()}.")

Epoch: 1, Loss : 0.5762595719747601.
Epoch: 2, Loss : 0.5591814103354904.
Epoch: 3, Loss : 0.5427355436426077.
Epoch: 4, Loss : 0.5268992761388678.
Epoch: 5, Loss : 0.5116508670627802.
Epoch: 6, Loss : 0.4969695067175949.
Epoch: 7, Loss : 0.48283527277717675.
Epoch: 8, Loss : 0.46922908894819293.
Epoch: 9, Loss : 0.456132698074045.
Epoch: 10, Loss : 0.4435286524865376.
Epoch: 11, Loss : 0.43140031877090357.
Epoch: 12, Loss : 0.41973189065687955.
Epoch: 13, Loss : 0.4079301703364243.
Epoch: 14, Loss : 0.3948260980122737.
Epoch: 15, Loss : 0.3822070103574496.
Epoch: 16, Loss : 0.37006027925738855.
Epoch: 17, Loss : 0.3583743422822195.
Epoch: 18, Loss : 0.347138523657533.
Epoch: 19, Loss : 0.3363427049988318.
Epoch: 20, Loss : 0.3259768366669907.


In [15]:
print(f"Model Weights:{model.weights}")

Model Weights:tensor([ 0.2437,  0.7019,  0.6392,  0.1924,  0.8427,  0.7495,  0.6253,  0.9574,
         0.2183,  0.1152,  0.1153,  0.4998,  0.0367,  0.2242,  0.3900,  0.5906,
        -0.0638,  0.1139,  0.5802,  0.2914,  0.6968,  0.6273,  0.8911,  0.1420,
         0.3002,  0.8107,  0.0851,  0.5021,  0.0794,  0.5968],
       dtype=torch.float64, requires_grad=True)


In [16]:
print(f"Model Bias:{model.bias}")

Model Bias:tensor([-0.0607], dtype=torch.float64, requires_grad=True)


In [17]:
#Evaluation of the Model

with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred>0.9).float()

    #Accuracy score of the ML NN Pipeline
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"The Accuracy is : {accuracy}")

The Accuracy is : 0.859649121761322
